In [1]:
import numpy as np
import matplotlib.pyplot as pp
import seaborn

In [2]:
%matplotlib inline

In [3]:
import urllib.request
urllib.request.urlretrieve('ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt','stations.txt')

('stations.txt', <email.message.Message at 0x1f557d8c208>)

In [4]:
open('stations.txt','r').readlines()[:10]

['ACW00011604  17.1167  -61.7833   10.1    ST JOHNS COOLIDGE FLD                       \n',
 'ACW00011647  17.1333  -61.7833   19.2    ST JOHNS                                    \n',
 'AE000041196  25.3330   55.5170   34.0    SHARJAH INTER. AIRP            GSN     41196\n',
 'AEM00041194  25.2550   55.3640   10.4    DUBAI INTL                             41194\n',
 'AEM00041217  24.4330   54.6510   26.8    ABU DHABI INTL                         41217\n',
 'AEM00041218  24.2620   55.6090  264.9    AL AIN INTL                            41218\n',
 'AF000040930  35.3170   69.0170 3366.0    NORTH-SALANG                   GSN     40930\n',
 'AFM00040938  34.2100   62.2280  977.2    HERAT                                  40938\n',
 'AFM00040948  34.5660   69.2120 1791.3    KABUL INTL                             40948\n',
 'AFM00040990  31.5000   65.8500 1010.0    KANDAHAR AIRPORT                       40990\n']

In [5]:
stations = {}

for line in open('stations.txt','r'):
    if 'GSN' in line:
        fields = line.split()
        
        stations[fields[0]] = ' '.join(fields[4:])

In [6]:
len(stations)

997

In [7]:
def findstation(s):
    found = {code: name for code,name in stations.items() if s in name}
    print(found)

In [8]:
findstation('LIHUE')

{'USW00022536': 'HI LIHUE WSO AP 1020.1 GSN 91165'}


In [9]:
findstation('SAN DIEGO')

{'USW00023188': 'CA SAN DIEGO LINDBERGH FLD GSN 72290'}


In [10]:
findstation('MINNEAPOLIS')

{'USW00014922': 'MN MINNEAPOLIS/ST PAUL AP GSN HCN 72658'}


In [11]:
findstation('IRKUTSK')

{'RSM00030710': 'IRKUTSK GSN 30710'}


In [12]:
datastations = ['USW00022536','USW00023188','USW00014922','RSM00030710']

In [13]:
open('USW00022536.dly','r').readlines()[:10]

['USW00022536195002TMAX  256  0  256  0  256  0  267  0  217  0  228  0  256  0  272  0  256  0  256  0  256  0  244  0  256  0  256  0  244  0  244  0  250  0  256  0  239  0  250  0  256  0  256  0  267  0  261  0  267  0  267  0  261  0  261  0-9999   -9999   -9999   \n',
 'USW00022536195002TMIN  178  0  156  0  161  0  167  0  167  0  167  0  189  0  211  0  206  0  217  0  217  0  211  0  200  0  200  0  206  0  183  0  206  0  206  0  206  0  194  0  206  0  200  0  206  0  200  0  211  0  183  0  172  0  200  0-9999   -9999   -9999   \n',
 'USW00022536195002PRCP    0  0    0  0    0  0    0  0  737  0  406  0   36  0   38  0    0T 0    0T 0    0  0    0T 0   18  0    5  0   10  0   18  0   15  0    5  0    0T 0    0T 0   23  0   10  0    3  0   48  0    0T 0    0T 0    0T 0    5  0-9999   -9999   -9999   \n',
 'USW00022536195002SNOW    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0

In [14]:
open('readme.txt','r').readlines()[98:121]

FileNotFoundError: [Errno 2] No such file or directory: 'readme.txt'

In [ ]:
def parsefile(filename):
    return np.genfromtxt(filename,
                         delimiter = dly_delimiter,
                         usecols = dly_usecols,
                         dtype = dly_dtype,
                         names = dly_names)

In [ ]:
dly_delimiter = [11,4,2,4] + [5,1,1,1] * 31
dly_usecols = [1,2,3] + [4*i for i in range(1,32)]
dly_dtype = [np.int32,np.int32,(np.str_,4)] + [np.int32] * 31
dly_names = ['year','month','obs'] + [str(day) for day in range(1,31+1)]

In [ ]:
lihue = parsefile('USW00022536.dly')

In [ ]:
lihue

In [ ]:
def unroll(record):
    startdate = np.datetime64('{}-{:02}'.format(record['year'],record['month']))
    dates = np.arange(startdate,startdate + np.timedelta64(1,'M'),np.timedelta64(1,'D'))
    
    rows = [(date,record[str(i+1)]/10) for i,date in enumerate(dates)]
    
    return np.array(rows,dtype=[('date','M8[D]'),('value','d')])

In [ ]:
unroll(lihue[0])

In [ ]:
def getobs(filename,obs):
    return np.concatenate([unroll(row) for row in parsefile(filename) if row[2] == obs])

In [ ]:
getobs('USW00022536.dly','TMIN')

In [ ]:
lihue_tmax = getobs('USW00022536.dly','TMAX')
lihue_tmin = getobs('USW00022536.dly','TMIN')

In [ ]:
pp.plot(lihue_tmax['date'],lihue_tmax['value'])

In [ ]:
def getobs(filename,obs):
    data = np.concatenate([unroll(row) for row in parsefile(filename) if row[2] == obs])
    
    data['value'][data['value'] == -999.9] = np.nan
    
    return data

In [ ]:
lihue_tmax = getobs('USW00022536.dly','TMAX')
lihue_tmin = getobs('USW00022536.dly','TMIN')

In [ ]:
pp.plot(lihue_tmax['date'],lihue_tmax['value'])
pp.plot(lihue_tmin['date'],lihue_tmin['value'])

In [ ]:
np.mean(lihue_tmin['value']), np.mean(lihue_tmax['value'])

In [ ]:
x = np.array([1,3,5,7],'d')
y = np.array([10,5,2,7],'d')

pp.plot(x,y,'o')

pp.axis([0,8,0,12])

In [ ]:
xs = np.linspace(1,7)

In [ ]:
ys = np.interp(xs,x,y)

In [ ]:
pp.plot(xs,ys,'.')
pp.plot(x,y,'o')

pp.axis([0,8,0,12])

In [ ]:
def fillnans(data):
    nan = np.isnan(data['value'])
    
    data['value'][nan] = np.interp(data['date'][nan],data['date'][~nan],data['value'][~nan])

In [ ]:
# fillnans(lihue_tmax)

In [ ]:
def fillnans(data):
    dates_float = data['date'].astype(np.float64)
    
    nan = np.isnan(data['value'])
    
    data['value'][nan] = np.interp(dates_float[nan],dates_float[~nan],data['value'][~nan])

In [ ]:
fillnans(lihue_tmax)
fillnans(lihue_tmin)

In [ ]:
np.mean(lihue_tmin['value']), np.mean(lihue_tmax['value'])

In [ ]:
pp.plot(lihue_tmin['date'],lihue_tmin['value'])

In [ ]:
def plot_smoothed(t,win=10):
    smoothed = np.correlate(t['value'],np.ones(win)/win,'same')
    
    pp.plot(t['date'],smoothed)

In [ ]:
pp.plot(lihue_tmin[10000:12000]['date'],lihue_tmin[10000:12000]['value'])

plot_smoothed(lihue_tmin[10000:12000])
plot_smoothed(lihue_tmin[10000:12000],30)

In [ ]:
pp.figure(figsize=(10,6))

for i,code in enumerate(datastations):
    pp.subplot(2,2,i+1)
    
    plot_smoothed(getobs('{}.dly'.format(code),'TMIN'),365)
    plot_smoothed(getobs('{}.dly'.format(code),'TMAX'),365)
    
    pp.title(stations[code])
    pp.axis(xmin=np.datetime64('1952'),xmax=np.datetime64('2012'),ymin=-10,ymax=30)

pp.tight_layout()

In [ ]:
def selectyear(data,year):
    start = np.datetime64('{}'.format(year))
    end = start + np.timedelta64(1,'Y')
    
    return data[(data['date'] >= start) & (data['date'] < end)]['value']

In [ ]:
selectyear(lihue_tmin,1951)

In [ ]:
lihue_tmin_all = np.vstack([selectyear(lihue_tmin,year)[:365] for year in range(1951,2014+1)])

In [ ]:
lihue_tmin_all

In [ ]:
lihue_tmin_all.shape

In [ ]:
lihue_tmin_recordmin = np.min(lihue_tmin_all,axis=0)
lihue_tmin_recordmax = np.max(lihue_tmin_all,axis=0)

In [ ]:
pp.plot(lihue_tmin_recordmax,'.')

In [ ]:
lihue_tmax_all = np.vstack([selectyear(lihue_tmax,year)[:365] for year in range(1951,2014+1)])

In [ ]:
pp.figure(figsize=(12,4))

days = np.arange(1,365+1)

pp.fill_between(days,np.min(lihue_tmin_all,axis=0),np.max(lihue_tmin_all,axis=0),alpha=0.4)
pp.plot(selectyear(lihue_tmin,2009))

pp.fill_between(days,np.min(lihue_tmax_all,axis=0),np.max(lihue_tmax_all,axis=0),alpha=0.4)
pp.plot(selectyear(lihue_tmax,2009))

pp.axis(xmax=365)

In [ ]:


minn_tmax = getobs('USW00014922.dly','TMAX')
minn_tmin = getobs('USW00014922.dly','TMIN')

fillnans(minn_tmax)
fillnans(minn_tmin)

minn_mean_tmax_byyear = [np.mean(selectyear(minn_tmax,year)) for year in range(1951,2014+1)] 

year = np.arange(1951,2014+1,1)
pp.plot(year,minn_mean_tmax_byyear)

# minn_highest_mean_tmax = np.max(minn_mean_tmax_byyear)
# for year in range(0,2014-1951):
#     if minn_mean_tmax_byyear[year]==minn_highest_mean_tmax:
#         print(1951+year)

print(f'Warmest year was {1951+np.argmax(minn_mean_tmax_byyear)}')


In [ ]:


sd_tmin = getobs('USW00023188.dly','TMIN')
sd_tmax = getobs('USW00023188.dly','TMAX')

fillnans(sd_tmin)
fillnans(sd_tmax)

sd_tmin_all = np.vstack([selectyear(sd_tmin,year)[:365] for year in range(1951,2014+1)])

sd_mean_tmin_byyear = np.mean(sd_tmin_all, axis=1)

year = np.arange(1951,2014+1,1)
pp.plot(year,sd_mean_tmin_byyear)

# sd_lowest_mean_tmin = np.min(sd_mean_tmin_byyear)
# for year in range(0,2014-1951):
#     if sd_mean_tmin_byyear[year]==sd_lowest_mean_tmin:
#         print(1951+year)

print(f'Coldest year was {1951+np.argmin(sd_mean_tmin_byyear)}')

In [ ]:
pp.figure(figsize=(12,6))
days = np.arange(1,366+1)

pp.fill_between(days,selectyear(sd_tmin,1964),selectyear(sd_tmax,1964),alpha=0.4)

pp.fill_between(days,selectyear(minn_tmin,2012),selectyear(minn_tmax,2012),alpha=0.4)
pp.axis(xmax=366)
